## 1. Import packages and data, define utility functions

In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import collections
import pickle

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
hse_path = "/Users/jdjumalieva/Downloads/UKDA-8860-tab/"

In [3]:
proj_path = "/Users/jdjumalieva/Documents/Analysis/weight_loss_modelling/"

#### Utility functions

In [4]:
def assign_bmi_class(val):
    '''Generates a BMI class label based on the provided BMI.
    '''
    if val <= 18.5:
        res = "underweight"
    elif val > 18.5 and val < 25:
        res = "normal"
    elif val >= 25 and val < 30:
        res = "overweight"
    elif val >= 30:
        res = "obese"
    return res

In [5]:
def recode_age(val):
    '''Recodes age from a numeric code to a string label.
    '''
    if val == 7:
        res = "16-19"
    elif val == 8:
        res = "20-24"
    elif val == 9:
        res = "25-29"
    elif val == 9:
        res = "25-29"
    elif val == 10:
        res = "30-34"
    elif val == 11:
        res = "35-39"
    elif val == 12:
        res = "40-44"
    elif val == 13:
        res = "45-49"
    elif val == 14:
        res = "50-54"
    elif val == 15:
        res = "55-59"
    elif val == 16:
        res = "60-64"
    elif val == 17:
        res = "65-69"
    elif val == 18:
        res = "70-74"
    elif val == 19:
        res = "75-79"
    elif val == 20:
        res = "80-84"
    elif val == 21:
        res = "85-89"
    elif val == 22:
        res = "90+"
    else:
        res = "Na"
    return res

In [6]:
# Recode sex
def recode_sex(val):
    '''Recodes sex from a numeric code to a string label.
    '''
    if val == 1:
        res = "male"
    elif val == 2:
        res = "female"
    return res

In [7]:
# Recode ethnicity
def recode_ethnicity(val):
    '''Recodes ethnicity from a numeric code to a string label.
    '''
    if val == 1:
        res = "White"
    elif val == 2:
        res = "Black"
    elif val == 3:
        res = "Asian"
    elif val == 4:
        res = "Mixed"
    elif val == 5:
        res = "Other"
    else:
        res = "Na"
    return res

In [8]:
def midpoint_age(val):
    '''Returns a mid point for a provided age range.
    '''
    unpack_vals = val.split("-")
    if unpack_vals == ["90+"]:
        res = 90
    else:
        unpack_vals = [int(elem) for elem in unpack_vals]
        res = sum(unpack_vals) / len(unpack_vals)
    return int(round(res, 0))

#### Subset and preprocess HSE data

In [9]:
hse = pd.read_csv(os.path.join(hse_path, "tab/hse_2019_eul_20211006.tab"), sep="\t")

In [10]:
hse.head()

,SerialA,HSEYR,SampType,FinOutc,hhsize6,Nofad3,Nofch3,Qrtint,intdayw,IndOut,...,qimd19,urban14b,PSU_SCR,cluster194,cluster94,cluster48,wt_int,wt_nurse,wt_blood,wt_cotinine
0,2902917,2019,1,214,2,2,0,1,4,110,...,5,1,2191163,219809,219508,219702,0.902759,,,
1,2907544,2019,1,110,1,1,0,1,7,110,...,4,1,2191163,219809,219508,219702,0.949745,,,
2,2902083,2019,1,110,6,2,3,1,4,110,...,4,1,2191163,219809,219508,219702,1.129456,1.40038494921487,1.51514388499197,
3,2904689,2019,1,110,6,2,3,1,4,110,...,4,1,2191163,219809,219508,219702,0.939781,1.16449825799688,,
4,2905230,2019,1,110,6,2,3,1,4,110,...,4,1,2191163,219809,219508,219702,1.862802,2.17515016111921,,


In [11]:
# [print(c) for c in hse.columns]

In [12]:
# Here we define a small subset of columsn that are useful for our analysis.
hse_subset = hse[
    [
        "SerialA",
        "Sex",
        "ag16g10",
        "age16g5",
        "Age35g",
        "Ag015g4",
        "nssec8",
        "origin2",
        "eqv5",
        "qimd19",
        "RelHite",
        "RelWaitB",
        "Height",
        "Weight",
        "HtM17",
        "WtM17",
        "EstHt2",
        "EstWt2",
        "BMI",
        "BMIsr",
        "BMIOwgt",
        "BMIVal",
        "BMIVal2",
        "wt_int",
        'marstatD', 
        'tenureb', 
        'HHINC3', 
        'Activb2', 
        'HRPSOC10B', 
        'GOR1', 
        'topqual3', 
        'hpnssec8'
    ]
]

In [13]:
hse_subset.head()

,SerialA,Sex,ag16g10,age16g5,Age35g,Ag015g4,nssec8,origin2,eqv5,qimd19,...,BMIVal2,wt_int,marstatD,tenureb,HHINC3,Activb2,HRPSOC10B,GOR1,topqual3,hpnssec8
0,2902917,2,5,11,16,-1,6,2,3,5,...,28.877782,0.902759,6,2,-1,2,61,7,3,6
1,2907544,1,4,8,13,-1,7,3,-1,4,...,-1.000000,0.949745,2,2,-1,2,82,7,7,7
2,2902083,1,3,6,11,-1,7,3,1,4,...,22.867883,1.129456,2,4,-1,2,82,7,4,7
3,2904689,2,2,4,9,-1,8,3,1,4,...,30.120438,0.939781,2,4,-1,9,82,7,7,7
4,2905230,2,-1,-1,2,1,-1,3,1,4,...,-1.000000,1.862802,-1,4,-1,-1,82,7,-1,7


In [14]:
# Checking for missing values
hse_subset.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10299 entries, 0 to 10298
Data columns (total 32 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SerialA    10299 non-null  int64  
 1   Sex        10299 non-null  int64  
 2   ag16g10    10299 non-null  int64  
 3   age16g5    10299 non-null  int64  
 4   Age35g     10299 non-null  int64  
 5   Ag015g4    10299 non-null  int64  
 6   nssec8     10299 non-null  int64  
 7   origin2    10299 non-null  int64  
 8   eqv5       10299 non-null  int64  
 9   qimd19     10299 non-null  int64  
 10  RelHite    10299 non-null  int64  
 11  RelWaitB   10299 non-null  int64  
 12  Height     10299 non-null  float64
 13  Weight     10299 non-null  float64
 14  HtM17      10299 non-null  float64
 15  WtM17      10299 non-null  float64
 16  EstHt2     10299 non-null  float64
 17  EstWt2     10299 non-null  float64
 18  BMI        10299 non-null  float64
 19  BMIsr      10299 non-null  float64
 20  BMIOwg

In [15]:
# Subset adults only
adults = hse_subset[hse_subset["Age35g"] >= 7]

In [16]:
# Drop missing data on weight and height
adults = adults[(adults["EstWt2"] != -1.0) & (adults["EstHt2"] != -1.0)]

## 2. Preprocess and aggregate data

1. Calculate BMI from estimates

[weight (kg) / height (cm) / height (cm)] x 10,000 [from CDC resources](https://www.cdc.gov/nccdphp/dnpao/growthcharts/training/bmiage/page5_1.html)

2. Assign BMI class

3. Weight measurements using provided sample weights

4. Group adults by:
- Gender (1 = Male, 2 = Female)
- Age
- BMI class
- Ethnicity (1 = White, 2 = Black, 3 = Asian, 4 = Mixed, 5 = Other) (park for now)
- Deprivation (park for now)

5. Aggregate:
- Sum of weights
- Sum of weighted weight
- Sum of weighted BMI

6. Calculate:
- Weighted average weight
- Weighted average BMI

In [17]:
# Calculate BMI
height_sq = adults["EstHt2"] * adults["EstHt2"] / 10000
adults["BMI_est"] = adults["EstWt2"] / height_sq

In [18]:
# Assign BMI class
adults["BMI_class"] = adults["BMI_est"].apply(lambda x: assign_bmi_class(x))

In [19]:
# Produce string labels for age
adults["age_group"] = adults["Age35g"].apply(lambda x: recode_age(x))

In [20]:
# Recode sex
adults["sex"] = adults["Sex"].apply(lambda x: recode_sex(x))

In [21]:
# Recode ethnicity
adults["ethnicity"] = adults["origin2"].apply(lambda x: recode_ethnicity(x))

In [22]:
# Drop 5 observations without ethnicity
adults = adults[adults["ethnicity"] != "Na"]

In [23]:
# Calculate mid point for age groups
adults["Age_est"] = adults["age_group"].apply(lambda x: midpoint_age(x))

In [25]:
adults = adults[
    [
        "wt_int",
        "BMI_est",
        "BMI_class",
        "age_group",
        "sex",
        "ethnicity",
        "EstWt2",
        "EstHt2",
        "Age_est",
        "qimd19",
        'marstatD', 
        'tenureb', 
        'HHINC3', 
        'Activb2', 
        'HRPSOC10B', 
        'GOR1', 
        'topqual3', 
        'hpnssec8'
    ]
]

In [26]:
# Reorder columns
adults = adults[
    [
        "sex",
        "age_group",
        "Age_est",
        "ethnicity",
        "BMI_class",
        "wt_int",
        "BMI_est",
        "EstWt2",
        "EstHt2",
        "qimd19",
        'marstatD', 
        'tenureb', 
        'HHINC3', 
        'Activb2', 
        'HRPSOC10B', 
        'GOR1', 
        'topqual3', 
        'hpnssec8'
    ]
]

# Rename some columns to make more informative
adults.columns = [
    "Sex",
    "Age_group",
    "Age_est",
    "Ethnicity",
    "BMI_class",
    "sample_weight",
    "BMI_est",
    "Wt_est",
    "Ht_est",
    "IMD_q",
    'marstatD', 
        'tenureb', 
        'HHINC3', 
        'Activb2', 
        'HRPSOC10B', 
        'GOR1', 
        'topqual3', 
        'hpnssec8'
]

In [27]:
adults.head()

,Sex,Age_group,Age_est,Ethnicity,BMI_class,sample_weight,BMI_est,Wt_est,Ht_est,IMD_q,marstatD,tenureb,HHINC3,Activb2,HRPSOC10B,GOR1,topqual3,hpnssec8
0,female,60-64,62,Black,overweight,0.902759,28.877782,67.6,153.00,5,6,2,-1,2,61,7,3,6
1,male,45-49,47,Asian,normal,0.949745,18.503251,52.0,167.64,4,2,2,-1,2,82,7,7,7
2,male,35-39,37,Asian,normal,1.129456,22.867883,65.7,169.50,4,2,4,-1,2,82,7,4,7
3,female,25-29,27,Asian,obese,0.939781,30.120438,83.2,166.20,4,2,4,-1,9,82,7,7,7
6,female,80-84,82,Black,overweight,0.890691,29.010722,68.0,153.10,3,5,1,-1,8,34,7,-1,4


In [28]:
adults.to_csv(os.path.join(proj_path, "calorie_deficit_scenarios_w_imd.csv"))
# adults.to_csv(os.path.join(proj_path, "calorie_deficit_scenarios_extended.csv"))

In [42]:
# Weight all the measures if you need to produce subgroup summaries

adults["wt_bmi_est"] = adults["BMI_est"] * adults["sample_weight"]
adults["wt_weight_est"] = adults["Wt_est"] * adults["sample_weight"]
adults["wt_height_est"] = adults["Wt_est"] * adults["sample_weight"]

#### Group HSE data and aggregate estimates

In [43]:
# Group and aggregate
adults.sort_values(
    ["Sex", "Age_group", "BMI_class"], inplace=True
)  # "ethnicity", "qimd19"

In [44]:
grouped_adults = adults.groupby(
    ["Sex", "Age_group", "BMI_class"]
)  # "ethnicity", "qimd19"

In [45]:
# grouped_adults.get_group(('female', '16-19', 'normal'))

In [46]:
weighted_sums = grouped_adults.agg(
    {"sample_weight": sum, "wt_bmi_est": sum, "wt_weight_est": sum, "wt_height_est": sum}
)

In [48]:
weighted_sums["BMI_est"] = weighted_sums["wt_bmi_est"] / weighted_sums["sample_weight"]
weighted_sums["Wt_est"] = weighted_sums["wt_weight_est"] / weighted_sums["sample_weight"]
weighted_sums["Ht_est"] = weighted_sums["wt_height_est"] / weighted_sums["sample_weight"]

In [49]:
weighted_sums.head()

sample_weight   wt_bmi_est  wt_weight_est  \
Sex    Age_group BMI_class                                                
female 16-19     normal          133.356337  2892.071838    7739.798609   
                 obese            25.232897   903.722963    2477.003550   
                 overweight       52.080014  1418.372593    3798.765013   
                 underweight      19.056841   326.877640     847.683382   
       20-24     normal          151.535788  3333.022761    9032.104869   

                              wt_height_est    BMI_est     Wt_est     Ht_est  
Sex    Age_group BMI_class                                                    
female 16-19     normal         7739.798609  21.686797  58.038476  58.038476  
                 obese          2477.003550  35.815268  98.165643  98.165643  
                 overweight     3798.765013  27.234490  72.940937  72.940937  
                 underweight     847.683382  17.152772  44.481842  44.481842  
       20-24     normal         9032.104869  21.994955  59.603774  59.603774

In [50]:
weighted_sums.to_csv(os.path.join(proj_path, "hse_grouped.csv"))

In [51]:
# Add information on BMI class share within each subgroup

hse_group_sizes = (
    weighted_sums.groupby(
        level=[
            0,
            1,
        ]
    )
    .sum()
    .reset_index()
)

In [52]:
hse_group_sizes = hse_group_sizes[["Sex", "Age_group", "sample_weight"]]
hse_group_sizes.columns = ["Sex", "Age_group", "group_sample_weight"]

In [53]:
weighted_sums_flat = weighted_sums.reset_index()

In [54]:
weighted_sums_flat["Age_est"] = weighted_sums_flat["Age_group"].apply(
    lambda x: midpoint_age(x)
)

In [55]:
weighted_sums_flat = weighted_sums_flat.merge(
    hse_group_sizes,
    how="left",
    left_on=["Sex", "Age_group"],
    right_on=["Sex", "Age_group"],
)

In [56]:
weighted_sums_flat.head()

,Sex,Age_group,BMI_class,sample_weight,wt_bmi_est,wt_weight_est,wt_height_est,BMI_est,Wt_est,Ht_est,Age_est,group_sample_weight
0,female,16-19,normal,133.356337,2892.071838,7739.798609,7739.798609,21.686797,58.038476,58.038476,18,229.726089
1,female,16-19,obese,25.232897,903.722963,2477.003550,2477.003550,35.815268,98.165643,98.165643,18,229.726089
2,female,16-19,overweight,52.080014,1418.372593,3798.765013,3798.765013,27.234490,72.940937,72.940937,18,229.726089
3,female,16-19,underweight,19.056841,326.877640,847.683382,847.683382,17.152772,44.481842,44.481842,18,229.726089
4,female,20-24,normal,151.535788,3333.022761,9032.104869,9032.104869,21.994955,59.603774,59.603774,22,276.106377


In [57]:
weighted_sums_flat["BMI_class_share"] = (
    weighted_sums_flat["sample_weight"] / weighted_sums_flat["group_sample_weight"]
)

#### Prepare population group size data

In [59]:
# Underlying subpopulation size
pop_est = pd.read_csv("/Users/jdjumalieva/Documents/Analysis/2019_sex_age_imd.csv")

In [60]:
new_cols = [col.strip() for col in pop_est.columns]

In [61]:
pop_est.columns = new_cols

In [62]:
age_cols = [
    "<1",
    "01-04",
    "05-09",
    "10-14",
    "15-19",
    "20-24",
    "25-29",
    "30-34",
    "35-39",
    "40-44",
    "45-49",
    "50-54",
    "55-59",
    "60-64",
    "65-69",
    "70-74",
    "75-79",
    "80-84",
    "85-89",
    "90+",
]

In [63]:
pop_by_age = pop_est.groupby("Sex")[age_cols].sum()

In [64]:
pop_by_age_t = pop_by_age.transpose()

In [65]:
age_group_size = collections.defaultdict(dict)
for ix, row in pop_by_age_t.iterrows():
    fem = row["Female"]
    male = row["Male"]
    age_group_size[ix]["female"] = fem
    age_group_size[ix]["male"] = male

In [66]:
# Retrieve underlying population size
pop_size_series = []
for ix, row in weighted_sums_flat.iterrows():
    sex = row["Sex"]
    age = row["Age_group"]
    if age == "16-19":
        age2 = "15-19"
    else:
        age2 = age
    pop_size_series.append(age_group_size[age2][sex])

In [67]:
# Calculate BMI class size for each subgroup
weighted_sums_flat["pop_size"] = pop_size_series
weighted_sums_flat["BMI_class_size"] = (
    weighted_sums_flat["BMI_class_share"] * weighted_sums_flat["pop_size"]
)

In [68]:
weighted_sums_flat.to_csv(os.path.join(proj_path, "hse_w_pop_size.csv"))

In [69]:
with open(os.path.join(proj_path, "pop_size_dict.pkl"), "wb") as outfile:
    pickle.dump(age_group_size, outfile)